In [39]:
import networkx as nx
import random
import numpy as np
import pandas as pd

student 
 - > passed courses
 - > grades
 - > gpa
 - > interests
 - > semester
 
 -> passed_courses ['id'], grades{'id': grade} (if grade == F then failed), gpa, interests
 
constrains 
- > 3-5 courses per semester
- > must complete prereq before taking a course
- > retake failed courses




In [40]:
courses = nx.read_graphml("courses.graphml")

In [45]:
def get_gpa(grades):
    """Calculate GPA from a list of grades."""
    if not grades:
        return 0.0
    total_points = sum(grade * 3 for grade in grades)  # all grades are 3-credit hours
    return total_points / (3 * len(grades))

def get_open_courses(courses, student_courses):
    """Returns a list of open courses that a student can take."""
    open_courses = []
    for course_id in courses.nodes():
        if course_id in student_courses:
            continue 
        prerequisites = list(courses.predecessors(course_id))
        if all(prereq in student_courses for prereq in prerequisites):
            open_courses.append(course_id)
            
    return open_courses

def generate_students(n, courses):
    """Generate a list of students with random courses and GPA."""
    students = []
    interests = ['Data', 'AI', 'Software']
    for i in range(n):
        semester = random.randint(1, 7)
        min_courses, max_courses = (3*semester, 5*semester)
        num_courses = random.randint(min_courses, max_courses)
        passed = []
        grades = []
        for _ in range(num_courses):
            c = random.choice(get_open_courses(courses, passed))
            grade = int(np.random.choice([0,1,2,3,4], p=[0.1,0.2,0.2,0.3,0.2])) # F, D, C, B, A
            if grade != 0:
                passed.append(c)
            grades.append({'id': c, 'grade': grade})
        gpa = float(get_gpa([grade['grade'] for grade in grades]))
        students.append({
            'id': f'{i+1}',
            'semester': semester,
            'courses': passed,
            'grades': grades,
            'gpa': gpa,
            'interests': random.sample(interests, k=random.randint(1, len(interests)))
        })
    return students


In [46]:
# simulate 100 students
students = generate_students(100, courses)

In [44]:
df = pd.DataFrame(students)
df.describe()

,semester,gpa
count,100.000000,100.000000
mean,3.860000,2.304015
std,1.853307,0.411369
min,1.000000,0.333333
25%,2.750000,2.062500
50%,4.000000,2.307692
75%,5.000000,2.500000
max,7.000000,3.500000


In [47]:
import json
with open('students.json', 'w') as f:
    json.dump(students, f, indent=4)

In [28]:
df.to_csv("students.csv", index=False)